In [175]:
##############
#K. David Roell CFPB 7/12/16
#Compiles a report of all macro edit failures for HMDA LAR data
#
##############
import codecs
import json
import os
import pandas as pd
import psycopg2
#from macro_sql import Q076 
#with open('macro_sql.json') as f: #FIXME change to quality_sql.json and refactor the rest of code
#    edit_sql = json.load(f)
macro_edits = ['Q006', 'Q007', 'Q008', 'Q009', 'Q010', 'Q011', 'Q015', 'Q016', 'Q023', 'Q031', 'Q047', 'Q048',
              'Q053', 'Q054', 'Q055', 'Q056', 'Q057', 'Q058', 'Q061', 'Q062', 'Q063', 'Q065', 'Q070', 'Q071',
              'Q072', 'Q073', 'Q074', 'Q075', 'Q076', 'Q080', 'Q081', 'Q082', 'Q083']
two_table_edits = ['Q011', 'Q070', 'Q071', 'Q072', 'Q075', 'Q076']
#conn = psycopg2.connect("dbname=hmdamaster user=roellk") #connect and return connection
#cur = conn.cursor()#instantiate cursor object to use in SQL queries
#parameter format for local use #consider changing hmdamaster to roellk if db changes cause a fail
params = {
'dbname':'hmdamaster',
'user':'roellk',
'password':'',
'host':'localhost',

}

#connect_string = "dbname=%s user=%s host=%s password =%s" %(dbname, user, host, password) #set a string for connection to SQL
try:
    conn = psycopg2.connect(**params)
    cur = conn.cursor()
    print("i'm connected")

except psycopg2.Error as e: #if database connection results in an error print the following
    print("I am unable to connect to the database: ", e)


lar_tables = ['hmdalar2014', 'hmdalar2013']
#edit_list = [item for item in dir(SQL_text) if not item.startswith("__")]

i'm connected


In [ ]:
first = True
for edit in macro_edits:
    SQL="" #instantiate empty string

    with open('SQL_files/'+edit+'.sql') as f: #cycle over all SQL files in edit list
        for line in f.readlines():
            SQL = SQL + line.replace('\n', ' ') #add lines from SQL file to string

    SQL = SQL[1:] #remove Byte Order Marker (BOM) from string
    print(edit) #check SQL output
    print() #format spacing
    #if edit in group with 2 tables, execute with 2 table parameters
    if edit in two_table_edits:
        print(SQL.format(table='hmdalar2014', table_prev='hmdalaro203'))
        cur.execute(SQL.format(table='HMDALAR2014', table_prev='HMDALAR2013')) #pass two tables to edits requiring it
        
    else:
        print(SQL.format(table='hmdalar2014'))
        cur.execute(SQL.format(table='HMDALAR2014')) #pass single table for most edits
    
    result_df = pd.DataFrame(cur.fetchall()) #convert query results to dataframe
    col_names = [desc[0] for desc in cur.description] #get list of column names from query result
    print(col_names)
    #print(result_df.head())
    print(len(result_df.columns))
#FIXME find out how to label empty df columns IE 58
#FIXME add in context cols to SQL like denom and numerator
    if len(result_df.columns) == len(col_names):
        print('setting col names')
        result_df.columns = col_names #change DF columns to result names
        
    if first == True and len(result_df.columns) >0:
        print('seeding frame')
        first = False
        macro_df = result_df.copy() #seed dataframe with first edit results
    elif first == False and len(result_df.columns) >0:
        print('merging frames')
        macro_df = macro_df.merge(result_df, how='outer', on=['arid', 'agency'])
    
    print(macro_df.head())

Q006

WITH numer AS (SELECT agency ,CONCAT(agency,rid) AS arid ,COUNT(sequence) as numer_count FROM hmdalar2014 WHERE action = '1' AND loan_purpose = '1' GROUP BY agency, CONCAT(agency, RID)),  denom AS (SELECT agency ,CONCAT(agency, RID) AS arid ,COUNT(sequence) as denom_count FROM hmdalar2014 WHERE loan_purpose = '1' GROUP BY agency, CONCAT(agency, rid))  SELECT numer.agency, numer.arid, (numer_count/denom_count::REAL) *100 AS Q006 FROM numer LEFT JOIN denom on numer.arid = denom.arid WHERE (numer_count/denom_count::REAL) * 100 > 95 AND numer_count > 25;
